In [1]:
import pandas as pd
import numpy as np
import cv2

In [2]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
skip = 0
face_data = []
file_name = input()
while (True):
    ret, frame = cap.read()
    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    faces = sorted(faces, key= lambda f:f[2]*f[3])
    for face in faces[-1:]:
        x, y, w, h = face
        cv2.rectangle(frame, (x, y), (x+w, y+h),(0, 255, 255), 5)
        offset = 20
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))
        skip += 1
        cv2.imshow('face_section', face_section)
        if skip%10 == 0:
            face_data.append(face_section)
#             print(len(face_data),'len')
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
face_data = np.array(face_data)
face_data = face_data.reshape((face_data.shape[0], -1))
np.save(file_name+'.npy', face_data)
print('data saved successfully at '+file_name+'.npy')
cap.release()
cv2.destroyAllWindows()

hru
data saved successfully at hru.npy


In [3]:
import os
import numpy as np

In [4]:
face_d = []
d = {}
mul = 0
for fx in os.listdir():
    if fx.endswith('.npy'):
        data_item = np.load(fx)
        y_column = mul*np.ones((len(data_item), 1))
        d[mul] = fx[:-4]
        mul+=1
        data_item = np.concatenate((data_item, y_column), axis = 1)
        face_d.append(data_item)

In [5]:
x = np.concatenate(face_d, axis = 0)
x_train = x[:, :-1]
y_train = x[:, -1]

In [6]:
# KNN algo
from collections import Counter

In [7]:
def predict_single(x_train, y_train, x, k):
    distances = []
    for i in range(len(x_train)):
        distance = ((x_train[i]-x)**2).sum()
        distances.append([distance, i])
    distances  = sorted(distances)
    target = []
    for i in range(k):
        index_corresponding_to_top_k = distances[i][1]
        target.append(y_train[index_corresponding_to_top_k])
    predicted_class = Counter(target).most_common(1)[0][0]
    return predicted_class

def predictKNN(x_train, y_train, x_test, k= 3):
    result = []
    for i in x_test:
        predicted_class = predict_single(x_train, y_train, i, k)
        result.append(predicted_class)
    return result

In [8]:
from sklearn.neighbors import KNeighborsClassifier
def predict_KNN(x_train, y_train, x_test):
    clf = KNeighborsClassifier(n_neighbors = 7)
    clf.fit(x_train, y_train)
    return clf.predict(x_test)

In [9]:
# testing data
import cv2

In [10]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    for face in faces:
        x, y, w, h = face
        offset = 20
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))
#         print(type(face_section))
        face_section = face_section.reshape(1, -1)
        predicted_class = predictKNN(x_train, y_train, face_section)
        predicted_name = d[predicted_class[0]]
        print(predicted_name)
        cv2.putText(frame, predicted_name, (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
    cv2.imshow('faces', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
hru
